<a href="https://colab.research.google.com/github/Oleksandr190378/data-computing/blob/main/Hm_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, Model
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.callbacks import  EarlyStopping
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix

In [28]:
from tensorflow.keras.datasets import fashion_mnist
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()

In [30]:
x_train = train_data
x_test = test_data
y_train = to_categorical(train_labels)
y_test = to_categorical(test_labels)

In [ ]:
data_augmentation = Sequential(
  [
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.2),
  ]
  )

In [5]:
inputs = keras.Input(shape=(28, 28, 1))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(32, 5, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
x = layers.MaxPooling2D(2)(x)
x = layers.Conv2D(64, 3, padding='same', use_bias=False)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.AveragePooling2D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model1 = keras.Model(inputs=inputs, outputs=outputs)

In [31]:
METRICS = ['accuracy',
           tf.metrics.Precision(name='precision'),
           tf.metrics.Recall(name='recall')]

In [7]:
model1.compile(optimizer="adam",
  loss="categorical_crossentropy",
  metrics=METRICS)

In [32]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=8, verbose=5, restore_best_weights=True)


In [9]:
history1 = model1.fit(
    x_train,
    y_train,
    epochs=50,
    batch_size=128,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping]
)

Epoch 1/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 73s 149ms/step - accuracy: 0.7225 - loss: 0.7777 - precision: 0.8477 - recall: 0.6134 - val_accuracy: 0.8597 - val_loss: 0.4602 - val_precision: 0.9359 - val_recall: 0.7538
Epoch 2/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 82s 148ms/step - accuracy: 0.8732 - loss: 0.3586 - precision: 0.9001 - recall: 0.8435 - val_accuracy: 0.8865 - val_loss: 0.3031 - val_precision: 0.9034 - val_recall: 0.8730
Epoch 3/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 83s 151ms/step - accuracy: 0.8901 - loss: 0.3021 - precision: 0.9112 - recall: 0.8695 - val_accuracy: 0.8940 - val_loss: 0.2851 - val_precision: 0.9089 - val_recall: 0.8826
Epoch 4/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 72s 154ms/step - accuracy: 0.9027 - loss: 0.2713 - precision: 0.9212 - recall: 0.8848 - val_accuracy: 0.8944 - val_loss: 0.2911 - val_precision: 0.9130 - val_recall: 0.8801
Epoch 5/50
469/469 ━━━━━━━━━━━━━━━━━━━━ 82s 153ms/step - accuracy: 0.9095 - loss: 0.2511 - precision: 0.9255 - recall: 0.8941 - val_accuracy: 0.9012

In [11]:
evaluation= model1.evaluate(x_test, y_test)
print(f'test accuracy - {evaluation[1]*100:.2f}%')

313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.9197 - loss: 0.2809 - precision: 0.9239 - recall: 0.9169
test accuracy - 92.13%


In [24]:
y_predict= model1.predict(x_test)
y_predict =np.argmax(y_predict, axis=1)
y_test =np.argmax(y_test, axis=1)
print(classification_report(y_test, y_predict))

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step
              precision    recall  f1-score   support

           0       0.86      0.87      0.86      1000
           1       0.99      0.99      0.99      1000
           2       0.85      0.90      0.87      1000
           3       0.93      0.92      0.92      1000
           4       0.89      0.87      0.88      1000
           5       0.99      0.98      0.99      1000
           6       0.79      0.76      0.77      1000
           7       0.95      0.98      0.97      1000
           8       0.98      0.99      0.98      1000
           9       0.98      0.96      0.97      1000

    accuracy                           0.92     10000
   macro avg       0.92      0.92      0.92     10000
weighted avg       0.92      0.92      0.92     10000



In [6]:
from keras.applications.vgg16 import VGG16

In [7]:
conv_base = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
conv_base.trainable = False


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [8]:
conv_base.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 32, 32, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 32, 32, 64)          │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 32, 32, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 16, 16, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 8, 8, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 8, 8, 256)           │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 4, 4, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 4, 4, 512)           │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 4, 4, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 2, 2, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 2, 2, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 1, 1, 512)           │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
import tensorflow as tf

In [26]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [11]:
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Rescaling(1./255)(inputs)
x = layers.Resizing(32, 32)(x)
x = layers.Lambda(lambda x: tf.repeat(x, 3, axis=-1), output_shape=(32, 32, 3))(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)
model_modified4 = keras.Model(inputs, outputs)

In [12]:
model_modified4.compile(optimizer = keras.optimizers.Adam(learning_rate=1e-5),
loss='categorical_crossentropy',
metrics=METRICS)

In [13]:
history_2 = model_modified4.fit(
    x_train,
    y_train,
    epochs=100,
    batch_size=64,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping]
)


Epoch 1/100
938/938 ━━━━━━━━━━━━━━━━━━━━ 70s 59ms/step - accuracy: 0.7461 - loss: 0.7736 - precision: 0.8755 - recall: 0.6471 - val_accuracy: 0.8872 - val_loss: 0.3230 - val_precision: 0.9109 - val_recall: 0.8670
Epoch 2/100
938/938 ━━━━━━━━━━━━━━━━━━━━ 61s 46ms/step - accuracy: 0.8989 - loss: 0.2900 - precision: 0.9181 - recall: 0.8803 - val_accuracy: 0.8978 - val_loss: 0.2858 - val_precision: 0.9110 - val_recall: 0.8851
Epoch 3/100
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 46ms/step - accuracy: 0.9134 - loss: 0.2425 - precision: 0.9272 - recall: 0.9001 - val_accuracy: 0.8986 - val_loss: 0.2859 - val_precision: 0.9103 - val_recall: 0.8903
Epoch 4/100
938/938 ━━━━━━━━━━━━━━━━━━━━ 44s 47ms/step - accuracy: 0.9260 - loss: 0.2066 - precision: 0.9382 - recall: 0.9163 - val_accuracy: 0.8836 - val_loss: 0.3234 - val_precision: 0.8926 - val_recall: 0.8772
Epoch 5/100
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 47ms/step - accuracy: 0.9344 - loss: 0.1863 - precision: 0.9439 - recall: 0.9268 - val_accuracy: 0.9025

In [14]:
evaluation2= model_modified4.evaluate(x_test, y_test)
print(f'test accuracy - {evaluation2[1]*100:.2f}%')

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9280 - loss: 0.3070 - precision: 0.9297 - recall: 0.9268
test accuracy - 93.00%


In [15]:
y_predict= model_modified4.predict(x_test)
y_predict =np.argmax(y_predict, axis=1)
y_test =np.argmax(y_test, axis=1)
print(classification_report(y_test, y_predict))

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
              precision    recall  f1-score   support

           0       0.90      0.85      0.87      1000
           1       0.99      0.98      0.99      1000
           2       0.92      0.89      0.90      1000
           3       0.94      0.92      0.93      1000
           4       0.91      0.88      0.89      1000
           5       0.99      0.98      0.99      1000
           6       0.74      0.85      0.79      1000
           7       0.97      0.98      0.98      1000
           8       0.99      0.99      0.99      1000
           9       0.98      0.98      0.98      1000

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000



In [33]:
conv_base3 = keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
conv_base3.trainable = True
for layer in conv_base.layers[:-5]:
  layer.trainable = False

In [35]:
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Rescaling(1./255)(inputs)
x = layers.Resizing(32, 32)(x)
x = layers.Lambda(lambda x: tf.repeat(x, 3, axis=-1), output_shape=(32, 32, 3))(x)
x = conv_base3(x)
x = layers.Flatten()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(10, activation='softmax')(x)
model_modified5 = keras.Model(inputs, outputs)

In [36]:
model_modified5.compile(optimizer = keras.optimizers.Adam(learning_rate=1e-5),
loss='categorical_crossentropy',
metrics=METRICS)

In [37]:
history_1 = model_modified5.fit(
    x_train,
    y_train,
    epochs=80,
    batch_size=64,
    validation_data=(x_test, y_test),
    callbacks=[early_stopping]
)


Epoch 1/80
938/938 ━━━━━━━━━━━━━━━━━━━━ 56s 50ms/step - accuracy: 0.7379 - loss: 0.7963 - precision: 0.8663 - recall: 0.6373 - val_accuracy: 0.8927 - val_loss: 0.3046 - val_precision: 0.9107 - val_recall: 0.8754
Epoch 2/80
938/938 ━━━━━━━━━━━━━━━━━━━━ 72s 44ms/step - accuracy: 0.8998 - loss: 0.2877 - precision: 0.9191 - recall: 0.8822 - val_accuracy: 0.8992 - val_loss: 0.2843 - val_precision: 0.9150 - val_recall: 0.8865
Epoch 3/80
938/938 ━━━━━━━━━━━━━━━━━━━━ 83s 45ms/step - accuracy: 0.9165 - loss: 0.2366 - precision: 0.9308 - recall: 0.9036 - val_accuracy: 0.8967 - val_loss: 0.2894 - val_precision: 0.9109 - val_recall: 0.8867
Epoch 4/80
938/938 ━━━━━━━━━━━━━━━━━━━━ 81s 44ms/step - accuracy: 0.9231 - loss: 0.2173 - precision: 0.9357 - recall: 0.9114 - val_accuracy: 0.9053 - val_loss: 0.2845 - val_precision: 0.9158 - val_recall: 0.8955
Epoch 5/80
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 44ms/step - accuracy: 0.9309 - loss: 0.1909 - precision: 0.9416 - recall: 0.9217 - val_accuracy: 0.9203 - va

In [38]:
evaluation2= model_modified5.evaluate(x_test, y_test)
print(f'test accuracy - {evaluation2[1]*100:.2f}%')

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.9320 - loss: 0.2865 - precision: 0.9336 - recall: 0.9312
test accuracy - 93.19%


За допомогою прийомів донавчання та виділення ознак ми покращили accuracy , precision та recall моделі...
